In [1]:
import string

import nltk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

### Loading "spam_ham_dataset".

In [2]:
df = pd.read_csv('../dataset/spam_ham_dataset.csv')
df

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0
...,...,...,...,...
5166,1518,ham,Subject: put the 10 on the ft\r\nthe transport...,0
5167,404,ham,Subject: 3 / 4 / 2000 and following noms\r\nhp...,0
5168,2933,ham,Subject: calpine daily gas nomination\r\n>\r\n...,0
5169,1409,ham,Subject: industrial worksheets for august 2000...,0


In [3]:
df = df.drop(['Unnamed: 0'], axis = 1)
df

,label,text,label_num
0,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,spam,"Subject: photoshop , windows , office . cheap ...",1
4,ham,Subject: re : indian springs\r\nthis deal is t...,0
...,...,...,...
5166,ham,Subject: put the 10 on the ft\r\nthe transport...,0
5167,ham,Subject: 3 / 4 / 2000 and following noms\r\nhp...,0
5168,ham,Subject: calpine daily gas nomination\r\n>\r\n...,0
5169,ham,Subject: industrial worksheets for august 2000...,0


### Separating the subject from the message and removing escape sequence.

In [4]:
test_preview = df['text'][0]
test_preview

"Subject: enron methanol ; meter # : 988291\r\nthis is a follow up to the note i gave you on monday , 4 / 3 / 00 { preliminary\r\nflow data provided by daren } .\r\nplease override pop ' s daily volume { presently zero } to reflect daily\r\nactivity you can obtain from gas control .\r\nthis change is needed asap for economics purposes ."

In [5]:
def subject_division(text):
    email_subjects = []
    email_texts = []
    for i in text:
        current_subject = ""
        current_text = ""
        is_subject = True
        for ch in i:
            if ch != '\r' and is_subject is True:
                current_subject += ch
            else:
                is_subject = False
                current_text += ch

        current_subject = current_subject.replace('Subject: ', '')
        current_text = current_text.replace('\r\n', '')
        
        email_subjects.append(current_subject)
        email_texts.append(current_text)

    return (email_subjects, email_texts)

In [6]:
email_subjects, email_texts = subject_division(df['text'])

for i in range(5):
    print(f"email_subjects = {email_subjects[i]}")
    print(f"email_texts = {email_texts[i]}\n")

email_subjects = enron methanol ; meter # : 988291
email_texts = this is a follow up to the note i gave you on monday , 4 / 3 / 00 { preliminaryflow data provided by daren } .please override pop ' s daily volume { presently zero } to reflect dailyactivity you can obtain from gas control .this change is needed asap for economics purposes .

email_subjects = hpl nom for january 9 , 2001
email_texts = ( see attached file : hplnol 09 . xls )- hplnol 09 . xls

email_subjects = neon retreat
email_texts = ho ho ho , we ' re around to that most wonderful time of the year - - - neon leaders retreat time !i know that this time of year is extremely hectic , and that it ' s tough to think about anything past the holidays , but life does go on past the week of december 25 through january 1 , and that ' s what i ' d like you to think about for a minute .on the calender that i handed out at the beginning of the fall semester , the retreat was scheduled for the weekend of january 5 - 6 . but because o

### Label preparation.

Note:  
$spam = 1$  
$hum = 0$

In [7]:
df = df.drop(['label'], axis = 1)
df = df.rename(columns={'label_num': 'label'})
df

,text,label
0,Subject: enron methanol ; meter # : 988291\r\n...,0
1,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,"Subject: photoshop , windows , office . cheap ...",1
4,Subject: re : indian springs\r\nthis deal is t...,0
...,...,...
5166,Subject: put the 10 on the ft\r\nthe transport...,0
5167,Subject: 3 / 4 / 2000 and following noms\r\nhp...,0
5168,Subject: calpine daily gas nomination\r\n>\r\n...,0
5169,Subject: industrial worksheets for august 2000...,0


#### Making changes.

In [8]:
df['text'] = email_texts
df['subject'] = email_subjects
df = df[['subject', 'text', 'label']] # Changed the order of columns
df

,subject,text,label
0,enron methanol ; meter # : 988291,this is a follow up to the note i gave you on ...,0
1,"hpl nom for january 9 , 2001",( see attached file : hplnol 09 . xls )- hplno...,0
2,neon retreat,"ho ho ho , we ' re around to that most wonderf...",0
3,"photoshop , windows , office . cheap . main tr...",abasements darer prudently fortuitous undergon...,1
4,re : indian springs,this deal is to book the teco pvr revenue . it...,0
...,...,...,...
5166,put the 10 on the ft,the transport volumes decreased from 25000 to ...,0
5167,3 / 4 / 2000 and following noms,hpl can ' t take the extra 15 mmcf / d over th...,0
5168,calpine daily gas nomination,">>julie , as i mention earlier we hope to star...",0
5169,industrial worksheets for august 2000 activity,attached are the worksheets for august 2000 ac...,0


<hr>

### Text preprocessing.